# Importing libraries

In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import seaborn as sns 
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


# Input phase 

### (User inputs stock ticker symbol )

In [18]:
ticker = "AAPL"

# Data selection (Extracting data from Yahoo finance Api)

In [19]:
start_date = datetime.today() - timedelta(days=10*365)
end_date = datetime.today()

# Use yfinance library to extract historical data
data = yf.download(ticker, start=start_date, end=end_date)

# Convert the data to a Pandas dataframe
df = pd.DataFrame(data)

# Print the first 5 rows of the dataframe
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-05-13,16.125357,16.353571,16.125000,16.240713,14.119643,316948800
2013-05-14,16.208929,16.257143,15.791071,15.852143,13.781820,447118000
2013-05-15,15.684286,15.750000,15.084286,15.316071,13.315761,741613600
2013-05-16,15.115714,15.637500,14.960714,15.520714,13.493676,603204000
2013-05-17,15.680357,15.717500,15.393214,15.473571,13.452690,427904400


# Data Pre-processing (Dealing with missing/null values)

In [20]:
#Checking any null/missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2517 entries, 2013-05-13 to 2023-05-10
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2517 non-null   float64
 1   High       2517 non-null   float64
 2   Low        2517 non-null   float64
 3   Close      2517 non-null   float64
 4   Adj Close  2517 non-null   float64
 5   Volume     2517 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 137.6 KB


In [21]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2.517000e+03
mean,67.229521,67.993777,66.508193,67.284259,65.488996,1.553992e+08
std,50.650112,51.312802,50.032432,50.705917,51.248185,1.013405e+08
min,13.977143,14.295357,13.888214,14.063571,12.226839,3.519590e+07
25%,27.905001,28.120001,27.552500,27.942499,25.562685,8.796540e+07
50%,43.417500,43.750000,43.087502,43.522499,41.389565,1.239340e+08
75%,119.620003,121.000000,118.150002,119.489998,117.892616,1.902916e+08
max,182.630005,182.940002,179.119995,182.009995,180.683868,1.065523e+09


# Data transformation
##  (Transforming raw daily historical into technical indicators using pandas_ta library )

In [22]:
# Calculate Technical indicators
#Calculating simple and weighted moving average (Sma and Wma)
df["SMA"] = ta.sma(df["Close"], length=10)
df["WMA"] = ta.wma(df["Close"], length=7)

#Calculating Relative strength index (RSI)
df["RSI"] = ta.rsi(df["Close"], length=14)
df["MOM"] = ta.mom(df['Close'],length=14)

#Calculating Stochastic %K (STCK) and Stochastic %D (STCD)
df.ta.stoch(high=df["High"], low=df["Low"], close=df["Close"], k=14, d=3, fillna=True, append=True)

# Calculating Williams %R
df['lwr'] = calculate_lwr(df,14)

#Calculating the accumlation/distribution indicator (ADO)
df['ADO'] = ta.adosc(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], fast=3, slow=10)

#Calculating Commodity Channel Inde (CCI)
df['CCI'] = ta.cci(high = df['High'] , low=df['Low'], close=df['Close'], length=20)

# Print the last 20 rows of the transformed data
df.tail(20)


,Open,High,Low,Close,Adj Close,Volume,SMA,WMA,RSI,MOM,STOCHk_14_3_3,STOCHd_14_3_3,lwr,ADO,CCI
Date,,,,,,,,,,,,,,,
2023-04-13,161.630005,165.800003,161.419998,165.559998,165.559998,68445600,163.597000,162.813216,63.268361,6.630005,56.844747,54.608582,11.786360,5.144344e+07,92.753604
2023-04-14,164.589996,166.320007,163.820007,165.210007,165.210007,49386500,163.882001,163.310717,62.388323,4.960007,70.380669,57.7451,15.009112,5.081306e+07,104.229917
2023-04-17,165.089996,165.389999,164.029999,165.229996,165.229996,41516200,163.915001,163.828215,62.420474,6.949997,86.126492,71.117303,14.825051,5.603727e+07,88.772481
2023-04-18,166.100006,167.410004,165.649994,166.470001,166.470001,49923000,163.945001,164.603215,64.450551,8.820007,86.167758,80.89164,11.662565,5.199716e+07,119.408815
2023-04-19,165.800003,168.160004,165.539993,167.630005,167.630005,47720200,164.145001,165.603573,66.285397,6.860001,89.062607,87.118952,6.324564,5.465809e+07,120.668862
2023-04-20,166.089996,167.869995,165.559998,166.649994,166.649994,52456400,164.434001,166.158928,63.312286,4.289993,87.997891,87.742752,18.019199,4.983839e+07,99.596961
2023-04-21,165.050003,166.449997,164.490005,165.020004,165.020004,58337300,164.470001,166.097858,58.604038,0.120010,79.395366,85.485288,37.470138,3.481265e+07,58.268552
2023-04-24,165.000000,165.600006,163.889999,165.330002,165.330002,41949600,164.800002,165.938572,59.225082,-0.839996,70.246593,79.213283,33.770885,3.463349e+07,43.525662
2023-04-25,165.190002,166.309998,163.729996,163.770004,163.770004,48714100,165.097002,165.397502,54.771879,-1.860001,58.790793,69.477584,52.386597,1.639247e+07,28.376531


In [23]:
# Since package pandas_ta doesnt have a function to calculate LWR indicator so we created a function to calculate it
 
def calculate_lwr(data, lookback):
    high = data['High'].rolling(window=lookback).max()
    low = data['Low'].rolling(window=lookback).min()
    close = data['Close']
    lwr = ((high - close) / (high - low)) * 100
    return lwr


### Creating target column


In [24]:
def create_target_column(data):
    # Shift the closing column by one day to get tomorrow's closing price
    data['tomorrow_closing'] = data['Close'].shift(-1)

    # Compare today's closing price with tomorrow's closing price
    data['target'] = (data['tomorrow_closing'] > data['Close']).astype(int)

    return data


In [25]:
df = create_target_column(df)
df

,Open,High,Low,Close,Adj Close,Volume,SMA,WMA,RSI,MOM,STOCHk_14_3_3,STOCHd_14_3_3,lwr,ADO,CCI,tomorrow_closing,target
Date,,,,,,,,,,,,,,,,,
2013-05-13,16.125357,16.353571,16.125000,16.240713,14.119643,316948800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.852143,0
2013-05-14,16.208929,16.257143,15.791071,15.852143,13.781820,447118000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.316071,0
2013-05-15,15.684286,15.750000,15.084286,15.316071,13.315761,741613600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.520714,1
2013-05-16,15.115714,15.637500,14.960714,15.520714,13.493676,603204000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.473571,0
2013-05-17,15.680357,15.717500,15.393214,15.473571,13.452690,427904400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.818929,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-04,164.889999,167.039993,164.309998,165.789993,165.789993,81235400,166.733998,167.711067,54.159298,0.579987,57.64559,74.454548,63.177438,3.433230e+06,7.700410,173.570007,1
2023-05-05,170.979996,174.300003,170.759995,173.570007,173.570007,113316400,167.588998,169.202855,67.617321,8.340012,62.580247,65.414637,6.347789,2.087351e+07,199.481182,173.500000,0
2023-05-08,172.479996,173.850006,172.110001,173.500000,173.500000,55962800,168.405998,170.326784,67.425498,7.029999,74.506075,64.910638,6.956548,3.675335e+07,180.082429,171.770004,0


In [26]:
#storing today's data for the final prediction
last_day = df.tail(1)
last_day

,Open,High,Low,Close,Adj Close,Volume,SMA,WMA,RSI,MOM,STOCHk_14_3_3,STOCHd_14_3_3,lwr,ADO,CCI,tomorrow_closing,target
Date,,,,,,,,,,,,,,,,,
2023-05-10,173.020004,174.029999,171.901001,173.554993,173.554993,51999273,170.185498,171.717678,65.391848,6.904999,88.18837,83.642112,6.478351,3.106858e+07,140.522733,NaN,0


# Modeling (Data mining phase)

In [27]:
#Importing libraries related to data preparation and the modeling phase
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


In [28]:
#removing data with null values the first 20 days since some techincal indicators are calculated for the previous 20 days 
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2497 entries, 2013-06-10 to 2023-05-09
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Open              2497 non-null   float64
 1   High              2497 non-null   float64
 2   Low               2497 non-null   float64
 3   Close             2497 non-null   float64
 4   Adj Close         2497 non-null   float64
 5   Volume            2497 non-null   int64  
 6   SMA               2497 non-null   float64
 7   WMA               2497 non-null   float64
 8   RSI               2497 non-null   float64
 9   MOM               2497 non-null   float64
 10  STOCHk_14_3_3     2497 non-null   object 
 11  STOCHd_14_3_3     2497 non-null   object 
 12  lwr               2497 non-null   float64
 13  ADO               2497 non-null   float64
 14  CCI               2497 non-null   float64
 15  tomorrow_closing  2497 non-null   float64
 16  target            2497 n

In [29]:
df['STOCHk_14_3_3'] = df['STOCHk_14_3_3'].astype(float)
df['STOCHd_14_3_3'] = df['STOCHd_14_3_3'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2497 entries, 2013-06-10 to 2023-05-09
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Open              2497 non-null   float64
 1   High              2497 non-null   float64
 2   Low               2497 non-null   float64
 3   Close             2497 non-null   float64
 4   Adj Close         2497 non-null   float64
 5   Volume            2497 non-null   int64  
 6   SMA               2497 non-null   float64
 7   WMA               2497 non-null   float64
 8   RSI               2497 non-null   float64
 9   MOM               2497 non-null   float64
 10  STOCHk_14_3_3     2497 non-null   float64
 11  STOCHd_14_3_3     2497 non-null   float64
 12  lwr               2497 non-null   float64
 13  ADO               2497 non-null   float64
 14  CCI               2497 non-null   float64
 15  tomorrow_closing  2497 non-null   float64
 16  target            2497 n

C:\Users\EGYPT_LAPTOP\AppData\Local\Temp\ipykernel_21204\1420186470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STOCHk_14_3_3'] = df['STOCHk_14_3_3'].astype(float)
C:\Users\EGYPT_LAPTOP\AppData\Local\Temp\ipykernel_21204\1420186470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STOCHd_14_3_3'] = df['STOCHd_14_3_3'].astype(float)


In [142]:
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)
scaled_df

,Open,High,Low,Close,Adj Close,Volume,SMA,WMA,RSI,MOM,STOCHk_14_3_3,STOCHd_14_3_3,lwr,ADO,CCI,tomorrow_closing,target
0,0.011302,0.010337,0.010360,0.009593,0.008315,0.402743,0.008277,0.009293,0.322675,0.517036,0.328522,0.455632,0.748459,0.336704,0.492716,0.009318,0.0
1,0.009398,0.008998,0.009608,0.009318,0.008077,0.243531,0.008194,0.008901,0.304810,0.518469,0.291171,0.349839,0.801480,0.316893,0.435850,0.008168,0.0
2,0.010194,0.008678,0.009214,0.008168,0.007080,0.223260,0.007915,0.008310,0.236503,0.513330,0.154438,0.238875,0.973047,0.249961,0.393191,0.008970,1.0
3,0.008712,0.007808,0.008620,0.008970,0.007775,0.243259,0.007575,0.008066,0.313592,0.515487,0.155372,0.178865,0.745680,0.305568,0.365993,0.007713,0.0
4,0.009326,0.007628,0.008566,0.007713,0.006685,0.229704,0.007146,0.007605,0.243429,0.509032,0.106856,0.114987,0.945803,0.290577,0.316905,0.008128,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,0.922148,0.928726,0.927617,0.913306,0.921441,0.029059,0.929714,0.930622,0.569840,0.558257,0.760712,0.879804,0.427340,0.466315,0.656156,0.903422,0.0
2493,0.894813,0.905719,0.910368,0.903422,0.911587,0.044684,0.929189,0.928317,0.503230,0.531712,0.575429,0.745117,0.631774,0.458051,0.529780,0.949746,1.0
2494,0.930923,0.948768,0.949404,0.949746,0.957770,0.075821,0.934409,0.937342,0.687695,0.688957,0.625194,0.650330,0.063478,0.475228,0.789923,0.949329,0.0
2495,0.939817,0.946100,0.957575,0.949329,0.957355,0.020156,0.939397,0.944142,0.685066,0.662411,0.745464,0.645045,0.069565,0.490868,0.763609,0.939028,0.0


In [136]:
# Split the data into input features (X) and target variable (y)
X = scaled_df[['WMA','SMA','RSI','MOM','STOCHk_14_3_3','STOCHd_14_3_3','lwr','ADO','CCI']].values
y = scaled_df['target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=233)

# Reshape the input data for LSTM (assuming a time step of 1)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])





In [137]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [138]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(9, input_shape=(1, 9)))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))  # Add dropout layer with a dropout rate of 0.1
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))  # Add dropout layer with a dropout rate of 0.1
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(patience=500, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/1000
63/63 [==============================] - 4s 16ms/step - loss: 0.6929 - accuracy: 0.5208 - val_loss: 0.6916 - val_accuracy: 0.5360
Epoch 2/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6926 - accuracy: 0.5238 - val_loss: 0.6901 - val_accuracy: 0.5360
Epoch 3/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6920 - accuracy: 0.5238 - val_loss: 0.6909 - val_accuracy: 0.5360
Epoch 4/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6924 - accuracy: 0.5178 - val_loss: 0.6903 - val_accuracy: 0.5360
Epoch 5/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6922 - accuracy: 0.5238 - val_loss: 0.6903 - val_accuracy: 0.5360
Epoch 6/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6918 - accuracy: 0.5238 - val_loss: 0.6902 - val_accuracy: 0.5360
Epoch 7/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6919 - accuracy: 0.5233 - val_loss: 0.6902 - val_accuracy: 0.5360
Epoch

63/63 [==============================] - 0s 4ms/step - loss: 0.6871 - accuracy: 0.5473 - val_loss: 0.6915 - val_accuracy: 0.5340
Epoch 59/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6883 - accuracy: 0.5403 - val_loss: 0.6912 - val_accuracy: 0.5420
Epoch 60/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6874 - accuracy: 0.5483 - val_loss: 0.6925 - val_accuracy: 0.5280
Epoch 61/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6883 - accuracy: 0.5373 - val_loss: 0.6897 - val_accuracy: 0.5360
Epoch 62/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6868 - accuracy: 0.5403 - val_loss: 0.6907 - val_accuracy: 0.5360
Epoch 63/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6863 - accuracy: 0.5473 - val_loss: 0.6918 - val_accuracy: 0.5420
Epoch 64/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6872 - accuracy: 0.5403 - val_loss: 0.6909 - val_accuracy: 0.5300
Epoch 65/1000

63/63 [==============================] - 0s 5ms/step - loss: 0.6793 - accuracy: 0.5543 - val_loss: 0.6927 - val_accuracy: 0.5340
Epoch 116/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6793 - accuracy: 0.5643 - val_loss: 0.6931 - val_accuracy: 0.5300
Epoch 117/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6815 - accuracy: 0.5699 - val_loss: 0.6920 - val_accuracy: 0.5240
Epoch 118/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6788 - accuracy: 0.5633 - val_loss: 0.6923 - val_accuracy: 0.5320
Epoch 119/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6775 - accuracy: 0.5563 - val_loss: 0.6935 - val_accuracy: 0.5300
Epoch 120/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6793 - accuracy: 0.5674 - val_loss: 0.6940 - val_accuracy: 0.5180
Epoch 121/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6789 - accuracy: 0.5563 - val_loss: 0.6931 - val_accuracy: 0.5260
Epoch 1

Epoch 172/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6698 - accuracy: 0.5794 - val_loss: 0.7013 - val_accuracy: 0.5260
Epoch 173/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6706 - accuracy: 0.5704 - val_loss: 0.6975 - val_accuracy: 0.5340
Epoch 174/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.6690 - accuracy: 0.5789 - val_loss: 0.6984 - val_accuracy: 0.5400
Epoch 175/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6692 - accuracy: 0.5854 - val_loss: 0.7045 - val_accuracy: 0.5280
Epoch 176/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.6657 - accuracy: 0.5864 - val_loss: 0.7019 - val_accuracy: 0.5340
Epoch 177/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6682 - accuracy: 0.5729 - val_loss: 0.6993 - val_accuracy: 0.5240
Epoch 178/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6646 - accuracy: 0.5774 - val_loss: 0.6993 - val_accuracy:

Epoch 229/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.6599 - accuracy: 0.5939 - val_loss: 0.7051 - val_accuracy: 0.5120
Epoch 230/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6559 - accuracy: 0.6004 - val_loss: 0.7059 - val_accuracy: 0.5120
Epoch 231/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6514 - accuracy: 0.6024 - val_loss: 0.7099 - val_accuracy: 0.5200
Epoch 232/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6508 - accuracy: 0.6179 - val_loss: 0.7104 - val_accuracy: 0.5240
Epoch 233/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.6578 - accuracy: 0.5964 - val_loss: 0.7028 - val_accuracy: 0.5140
Epoch 234/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.6564 - accuracy: 0.5854 - val_loss: 0.7113 - val_accuracy: 0.5080
Epoch 235/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.6513 - accuracy: 0.6074 - val_loss: 0.7063 - val_accuracy:

...

Epoch 742/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4834 - accuracy: 0.7601 - val_loss: 0.9030 - val_accuracy: 0.5460
Epoch 743/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4578 - accuracy: 0.7842 - val_loss: 0.9404 - val_accuracy: 0.5480
Epoch 744/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4736 - accuracy: 0.7737 - val_loss: 0.9015 - val_accuracy: 0.5260
Epoch 745/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4729 - accuracy: 0.7666 - val_loss: 0.9195 - val_accuracy: 0.5420
Epoch 746/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4755 - accuracy: 0.7656 - val_loss: 0.9104 - val_accuracy: 0.5440
Epoch 747/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4745 - accuracy: 0.7626 - val_loss: 0.9331 - val_accuracy: 0.5160
Epoch 748/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4724 - accuracy: 0.7702 - val_loss: 0.9319 - val_accuracy:

Epoch 799/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4503 - accuracy: 0.7732 - val_loss: 0.9832 - val_accuracy: 0.5400
Epoch 800/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4479 - accuracy: 0.7867 - val_loss: 0.9905 - val_accuracy: 0.5380
Epoch 801/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4536 - accuracy: 0.7787 - val_loss: 0.9724 - val_accuracy: 0.5380
Epoch 802/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4539 - accuracy: 0.7702 - val_loss: 0.9864 - val_accuracy: 0.5240
Epoch 803/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4512 - accuracy: 0.7757 - val_loss: 0.9796 - val_accuracy: 0.5360
Epoch 804/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4359 - accuracy: 0.7892 - val_loss: 0.9716 - val_accuracy: 0.5460
Epoch 805/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4526 - accuracy: 0.7747 - val_loss: 0.9825 - val_accuracy:

Epoch 856/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4330 - accuracy: 0.7972 - val_loss: 1.0072 - val_accuracy: 0.5340
Epoch 857/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4334 - accuracy: 0.7892 - val_loss: 1.0247 - val_accuracy: 0.5360
Epoch 858/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4324 - accuracy: 0.7927 - val_loss: 1.0212 - val_accuracy: 0.5460
Epoch 859/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4273 - accuracy: 0.7882 - val_loss: 1.0005 - val_accuracy: 0.5500
Epoch 860/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4266 - accuracy: 0.8007 - val_loss: 1.0112 - val_accuracy: 0.5540
Epoch 861/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4257 - accuracy: 0.7907 - val_loss: 1.0099 - val_accuracy: 0.5380
Epoch 862/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4309 - accuracy: 0.7957 - val_loss: 0.9992 - val_accuracy:

Epoch 913/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4117 - accuracy: 0.8082 - val_loss: 1.0418 - val_accuracy: 0.5480
Epoch 914/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4183 - accuracy: 0.8002 - val_loss: 1.0532 - val_accuracy: 0.5300
Epoch 915/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4069 - accuracy: 0.8082 - val_loss: 1.0452 - val_accuracy: 0.5480
Epoch 916/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4229 - accuracy: 0.8022 - val_loss: 1.0631 - val_accuracy: 0.5600
Epoch 917/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4152 - accuracy: 0.8012 - val_loss: 1.0322 - val_accuracy: 0.5440
Epoch 918/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4203 - accuracy: 0.7947 - val_loss: 1.0400 - val_accuracy: 0.5440
Epoch 919/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4153 - accuracy: 0.7967 - val_loss: 1.0458 - val_accuracy:

Epoch 970/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4046 - accuracy: 0.8027 - val_loss: 1.0582 - val_accuracy: 0.5380
Epoch 971/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.3875 - accuracy: 0.8262 - val_loss: 1.1134 - val_accuracy: 0.5620
Epoch 972/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.3853 - accuracy: 0.8187 - val_loss: 1.1204 - val_accuracy: 0.5460
Epoch 973/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4056 - accuracy: 0.8202 - val_loss: 1.0932 - val_accuracy: 0.5680
Epoch 974/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4093 - accuracy: 0.8032 - val_loss: 1.1017 - val_accuracy: 0.5520
Epoch 975/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.4131 - accuracy: 0.8022 - val_loss: 1.0898 - val_accuracy: 0.5620
Epoch 976/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.3892 - accuracy: 0.8182 - val_loss: 1.1028 - val_accuracy:

# Evaluation

In [139]:
predictions = model.predict(X_test)
predictions

16/16 [==============================] - 1s 2ms/step


array([[8.31656978e-02],
       [2.17589051e-01],
       [9.92712557e-01],
       [6.53926730e-01],
       [9.59660828e-01],
       [9.90725279e-01],
       [6.74485683e-01],
       [8.50241423e-01],
       [8.39659393e-01],
       [5.85357428e-01],
       [9.99301016e-01],
       [5.32671750e-01],
       [5.41669369e-01],
       [2.95479834e-01],
       [3.80871445e-01],
       [8.68238270e-01],
       [1.85140595e-01],
       [2.55812973e-01],
       [2.43333176e-01],
       [6.88543797e-01],
       [8.76608908e-01],
       [7.60289252e-01],
       [6.38101459e-01],
       [3.57304662e-01],
       [9.52444553e-01],
       [3.85094583e-01],
       [8.30044031e-01],
       [5.45165479e-01],
       [4.33240861e-01],
       [9.89580929e-01],
       [1.22854136e-01],
       [8.01070213e-01],
       [5.28795421e-01],
       [9.90572214e-01],
       [4.73026447e-02],
       [6.37200654e-01],
       [2.57823706e-01],
       [3.68662238e-01],
       [6.12966239e-01],
       [4.32878166e-01],


In [141]:
new_predictions =[]
for instance in predictions:
    if instance>0.5:
        new_predictions.append(1)
    else:
        new_predictions.append(0) 
        
print(classification_report(y_test,new_predictions))  

              precision    recall  f1-score   support

         0.0       0.52      0.50      0.51       232
         1.0       0.58      0.60      0.59       268

    accuracy                           0.55       500
   macro avg       0.55      0.55      0.55       500
weighted avg       0.55      0.55      0.55       500



In [127]:
y_test


array([1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
       1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 0., 0.

In [39]:
from sklearn.metrics import classification_report


In [329]:
print(classification_report(y_test,new_predictions))

              precision    recall  f1-score   support

         0.0       0.50      0.50      0.50       125
         1.0       0.50      0.50      0.50       123

    accuracy                           0.50       248
   macro avg       0.50      0.50      0.50       248
weighted avg       0.50      0.50      0.50       248



In [56]:
predictions = model.predict(X_test)
predictions

16/16 [==============================] - 1s 2ms/step


array([[0.51169544],
       [0.49927318],
       [0.54308486],
       [0.51169544],
       [0.5000698 ],
       [0.51169544],
       [0.53209215],
       [0.51169544],
       [0.49927318],
       [0.53293866],
       [0.51169544],
       [0.5330683 ],
       [0.53293866],
       [0.54308486],
       [0.51169544],
       [0.5000829 ],
       [0.51169544],
       [0.54308486],
       [0.54308486],
       [0.51169544],
       [0.51169544],
       [0.51169544],
       [0.5330683 ],
       [0.53293866],
       [0.49291357],
       [0.49927318],
       [0.5330683 ],
       [0.54308486],
       [0.5000829 ],
       [0.51169544],
       [0.49291357],
       [0.5330683 ],
       [0.54308486],
       [0.51169544],
       [0.54308486],
       [0.54308486],
       [0.54308486],
       [0.5330862 ],
       [0.53293866],
       [0.51169544],
       [0.51169544],
       [0.54308486],
       [0.49291357],
       [0.54308486],
       [0.51169544],
       [0.51169544],
       [0.4999734 ],
       [0.511